In [2]:
!pip install opencv-python

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import cv2
from scipy.spatial import distance
from keras import layers
from keras import models
from keras import optimizers
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

In [15]:
''' path of mask and no_mask '''
mask_path = r"C:\Users\vikas\Downloads\Git Project\Face Mask Detection\Face Mask Dataset\Train\WithMask"
no_mask_path = r"C:\Users\vikas\Downloads\Git Project\Face Mask Detection\Face Mask Dataset\Train\WithoutMask"

In [29]:
''' Data Augmentation '''

train_ds = ImageDataGenerator(rescale = 1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

''' flow from directory '''
train_ds = train_ds.flow_from_directory(r'C:\Users\vikas\Downloads\Git Project\Face Mask Detection\Face Mask Dataset\Train', target_size=(150, 150),
                                        batch_size = 32, class_mode = "binary")

val_ds = ImageDataGenerator(rescale = 1./255)
val_ds = val_ds.flow_from_directory(r'C:\Users\vikas\Downloads\Git Project\Face Mask Detection\Face Mask Dataset\Validation', target_size=(150, 150),
                                    batch_size = 32, class_mode = "binary")


test_ds = ImageDataGenerator(rescale = 1./255)
test_ds = test_ds.flow_from_directory(r'C:\Users\vikas\Downloads\Git Project\Face Mask Detection\Face Mask Dataset\Test', target_size = (150, 150),
                                          class_mode = "binary", batch_size = 32)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [33]:
''' Vgg19 Model'''
model1 = VGG19(weights='imagenet',include_top=False,input_shape=(150,150,3))

''' unfreezing layers'''
for layer in model1.layers:
    layer.trainable = False

''' container'''
model = models.Sequential()
model.add(model1)

''' flattening '''
model.add(layers.Flatten())
''' classification layer '''
model.add(layers.Dense(1,activation='sigmoid'))

''' lets see how model looks '''
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 8193      
Total params: 20,032,577
Trainable params: 8,193
Non-trainable params: 20,024,384
_________________________________________________________________


In [34]:
''' compile the model '''
model.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(lr = 1e-4), metrics=["acc"])

In [35]:
''' training '''
history = model.fit(train_ds,steps_per_epoch=len(train_ds)//32, epochs=50,
                    validation_data=val_ds, validation_steps=len(val_ds)//32)

Epoch 1/50
9/9 [==============================] - 317s 35s/step - loss: 0.6891 - acc: 0.5174 - val_loss: 0.6331 - val_acc: 0.6313
Epoch 2/50
9/9 [==============================] - 315s 35s/step - loss: 0.6216 - acc: 0.6979 - val_loss: 0.5897 - val_acc: 0.7625
Epoch 3/50
9/9 [==============================] - 312s 35s/step - loss: 0.6085 - acc: 0.7049 - val_loss: 0.5126 - val_acc: 0.7588
Epoch 4/50
9/9 [==============================] - 261s 29s/step - loss: 0.5839 - acc: 0.7396 - val_loss: 0.4781 - val_acc: 0.7987
Epoch 5/50
9/9 [==============================] - 218s 24s/step - loss: 0.5418 - acc: 0.7978 - val_loss: 0.4792 - val_acc: 0.8625
Epoch 6/50
9/9 [==============================] - 222s 25s/step - loss: 0.5390 - acc: 0.7986 - val_loss: 0.4169 - val_acc: 0.8575
Epoch 7/50
9/9 [==============================] - 230s 26s/step - loss: 0.5235 - acc: 0.8056 - val_loss: 0.4366 - val_acc: 0.8725
Epoch 8/50
9/9 [==============================] - 256s 28s/step - loss: 0.4833 - acc: 0.86

In [36]:
epochs = range(1, len(acc) + 1)

''' plotting training accuracy and validation accuracy '''
plt.figure(figsize = (15, 6))
plt.plot(epochs, history.history["acc"], "bo", label = "Eğitim Başarısı")
plt.plot(epochs, history.history["val_acc"], "b", label = "Test Başarısı")
plt.title("Eğitim ve Test Başarısı")
plt.legend()

''' plotting training loss and validation loss '''
plt.figure(figsize = (15, 6))
plt.plot(epochs, history.history["loss"], "bo", label = "Eğitim Kaybı")
plt.plot(epochs, history.history["val_loss"], "b", label = "Doğrulama Kaybı")
plt.legend()

plt.show()

NameError: name 'acc' is not defined

In [37]:
''' evaluating the model '''
test_loss, test_acc = model.evaluate(test_ds, steps = 9)
print("Test ACC: ", round(test_acc, 2))

9/9 [==============================] - 49s 5s/step
Test ACC:  0.94


In [41]:
from os import listdir
from os.path import isfile, join

In [52]:
## Now Predict
predict_dir_path='C:\\Users\\vikas\\Downloads\\Git Project\\Face Mask Detection\\Face Mask Dataset\\Test\\Common\\'
onlyfiles = [f for f in listdir(predict_dir_path) if isfile(join(predict_dir_path, f))]
print(onlyfiles)

['191.png', '193.png', '218.png', '219.png', '224.png', '225.png', '230.png', '231.png', '344.png', '345.png', '346.png', '350.png', '351.png', '408.png', '595.png', '608.png', '609.png', 'passp.jpg', 'vikashPic.jpg']


In [53]:
# predicting images
from keras.preprocessing import image
mask_counter = 0 
withoutMaskt_counter  = 0
for file in onlyfiles:
    img = image.load_img(predict_dir_path+file, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0:
        print(file + ": " + 'With Mask')
        mask_counter += 1
    else:
        print(file + ": " + 'without Mask')
        withoutMaskt_counter += 1
print("Total mask :",mask_counter)
print("Total without :",withoutMaskt_counter)

191.png: without Mask
193.png: without Mask
218.png: without Mask
219.png: without Mask
224.png: without Mask
225.png: without Mask
230.png: without Mask
231.png: without Mask
344.png: without Mask
345.png: without Mask
346.png: without Mask
350.png: without Mask
351.png: without Mask
408.png: With Mask
595.png: With Mask
608.png: without Mask
609.png: With Mask
passp.jpg: without Mask
vikashPic.jpg: With Mask
Total mask : 4
Total without : 15
